In [6]:
import pandas as pd 
import numpy as np
from datetime import date, datetime, timedelta
import lxml.html
import json
import requests 
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
session=requests.Session()
session.verify=False
r = session.get('https://www.esperanza.mx/'.replace(' ',''), headers={
'Upgrade-Insecure-Requests': '1',
'Connection': 'keep-alive',
'Content-Type': 'application/x-www-form-urlencoded', \
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'} )
session.close()



In [9]:
root = lxml.html.fromstring(r.text)

In [69]:
from scrapy import Selector
r1=root.xpath("//script[contains(.,'var locationList = [];')]")[0].text

In [72]:
import re
values = re.findall(r'var.*?=\s*(.*?);', r1, re.DOTALL | re.MULTILINE)

In [273]:
data=[values[i].replace(' ','').replace('\r\n','').replace('"',"'").replace('{','').replace('}','').replace('ranch','').replace('&#193','') for i in range(0,len(values))]
selected= [i for i in data if i.startswith('BId')]
n_selected= [i for i in range(0,len(data)) if data[i].startswith('BId')]
State= [i for i in data if i.startswith('BStateId')]
n_State= [i for i in range(0,len(data)) if data[i].startswith('BStateId')]
City= [i for i in data if i.startswith('BCityId')]
n_City= [i for i in range(0,len(data)) if data[i].startswith('BCityId')]
Estados=[st.replace("'",'').split(',B')[1].split(':')[1] for st in State]
Ciudades=[st.replace("'",'').split(',B')[1].split(':')[1] for st in City]
from cmath import nan
Tabla=[]
for a in selected:
    variables=a.replace("'",'').split(',B')
    id='NaN'
    Lat='NaN'
    Lon='NaN'
    Name='NaN'
    Adress='NaN'
    Phone='NaN'
    for i in variables:
        if i.split(':')[0]=='BId':
            id=i.split(':')[1]
        elif i.split(':')[0]=='Latitude':
            Lat=i.split(':')[1]
        elif i.split(':')[0]=='Longitude':
            Lon=i.split(':')[1]
        elif i.split(':')[0]=='Name':
            Name=i.split(':')[1]
        elif i.split(':')[0]=='Address':
            Address=i.split(':')[1]
        elif i.split(':')[0]=='Phone':
            Phone=i.split(':')[1]
    Tabla=Tabla+[[id,Lat,Lon,Name,Address,Phone]]      
Tabla=pd.DataFrame(Tabla,columns=['id','Lat','Lon','Name','Adress','Phone'])
nsel=[];ciu=[];est=[]
for j in range(0,len(n_selected)):
    nsel=nsel+[n_selected[j]]
    ncit=[ik<n_selected[j] for ik in set(n_City)]
    nest=[ik<n_selected[j] for ik in set(n_State)]
    ciu=ciu+[Ciudades[np.where(ncit)[0][-1]]]
    est=est+[Estados[np.where(nest)[0][-1]]]
Tabla['nsel']=nsel
Tabla['estado']=est
Tabla['ciudad']=ciu

In [274]:
Tabla

,id,Lat,Lon,Name,Adress,Phone,nsel,estado,ciudad
0,158,19.341136,-99.209296,Av.Toluca,"Av.Toluca207-B,OlivardelosPadres,01780,",NaN,3,CDMX,Azcapotzalco
1,159,19.389665,-99.210673,SantaFe,"CaminoaSantaFe440-B,ElPara&#237",NaN,4,CDMX,Azcapotzalco
2,160,19.375537,-99.195824,AlfonsoXIII,"FranzHals51,AlfonsoXIII,01460,",NaN,5,CDMX,Azcapotzalco
3,161,19.366084,-99.198853,LomasdePlateros,"Tacuba1,MercedG&#243",NaN,6,CDMX,Azcapotzalco
4,162,19.370675,-99.216023,OlivardelConde,SantaLuc&#237,NaN,7,CDMX,Azcapotzalco
...,...,...,...,...,...,...,...,...,...
120,229,20.649028,-103.319498,R&#237,"16deSeptiembre1126,Mexicaltzingo,44180,Guadala...",NaN,160,Yucatán,Merida
121,360,21.126889,-86.877639,ARCOKABAH,"LaRiojas/n,Canc&#250",NaN,163,Yucatán,Merida
122,362,21.134059,-86.854516,AVLALUNA,"Tepich506,SinNombre,77533,Canc&#250",NaN,164,Yucatán,Merida
123,364,21.179892,-86.863155,CAIMANES(PROXIMAMENTE),"Av.Kabah226,SinNombre,77516,Canc&#250",NaN,165,Yucatán,Merida


In [275]:
Tabla.to_csv('Esperanza.csv')

In [229]:
n_City

[2,
 9,
 12,
 18,
 24,
 32,
 36,
 40,
 57,
 60,
 65,
 69,
 76,
 79,
 84,
 88,
 92,
 94,
 96,
 100,
 107,
 111,
 118,
 122,
 126,
 128,
 132,
 134,
 137,
 144,
 151,
 157,
 162,
 167]

In [261]:
type(Tabla.iloc[100].nsel)

numpy.int64

In [231]:
n_City

[2,
 9,
 12,
 18,
 24,
 32,
 36,
 40,
 57,
 60,
 65,
 69,
 76,
 79,
 84,
 88,
 92,
 94,
 96,
 100,
 107,
 111,
 118,
 122,
 126,
 128,
 132,
 134,
 137,
 144,
 151,
 157,
 162,
 167]

In [237]:
np.where(n_City<Tabla.iloc[100].nsel)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False])

In [259]:
Tabla.iloc[100].nsel

130